In [1]:
%pip install pandas plotly matplotlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -U nbformat

In [3]:
import json
import pandas as pd
import random
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

In [4]:
# UPDATE_PARTICIPANT_TO_ACTION,    // ACTION 참여자 수정(참여자 추가 및 삭제) 이벤트
# CREATE_PROJECT_PROGRESS_ACTION,  // ACTION 생성 이벤트
# DELETE_PROJECT_PROGRESS_ACTION,  // ACTION 삭제 이벤트
# DONE_PROJECT_PROGRESS_ACTION     // ACTION 상태 완료로 변경이벤트

In [5]:
# 2. 중요도에 따른 평균 작업 처리 기간(시간) → Line chart
# - userId별 분류 -> importance별 분류
# - status=="DONE"       -> endDate-startDate

In [6]:
LOG_DATA_PATH = "data\\final_user-actions_dummy.json"

# 중첩 구조 평탄화해서 읽기
with open(LOG_DATA_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)
    # data = [json.loads(line.strip()) for line in f if line.strip()]  # .jsonl일 때 읽어오는 방법
    
df = pd.json_normalize(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   event                 100 non-null    object
 1   userId                100 non-null    object
 2   username              100 non-null    object
 3   timestamp             100 non-null    object
 4   workspaceId           100 non-null    int64 
 5   details.participants  100 non-null    object
 6   details.state         100 non-null    object
 7   details.name          100 non-null    object
 8   details.importance    100 non-null    int64 
 9   details.startDate     47 non-null     object
 10  details.endDate       47 non-null     object
dtypes: int64(2), object(9)
memory usage: 8.7+ KB


In [7]:
# df = pd.to_datetime(df[['timestamp', 'details.startDate', 'details.endDate']], utc=True)
# df.info()

In [8]:
# unique() 체크
print("df['event'].unique(): ", df['event'].unique())
print("df['userId'].unique(): ", df['userId'].unique())
print("df['workspaceId'].unique(): ", df['workspaceId'].unique())

print("df['details.state'].unique(): ", df['details.state'].unique())
print("df['details.importance'].unique(): ", df['details.importance'].unique())

df['event'].unique():  ['CREATE_PROJECT_PROGRESS_ACTION' 'DONE_PROJECT_PROGRESS_ACTION']
df['userId'].unique():  ['3' '4' '1' '2' '5']
df['workspaceId'].unique():  [1]
df['details.state'].unique():  ['IN_PROGRESS' 'DONE' 'BEFORE']
df['details.importance'].unique():  [1 2 3 5 4]


In [9]:
# 유저별 actions 분류
df_user1 = df[df['userId'] == '1']
df_user2 = df[df['userId'] == '2']
df_user3 = df[df['userId'] == '3']
df_user4 = df[df['userId'] == '4']
df_user5 = df[df['userId'] == '5']

user_df_list = [df_user1, df_user2, df_user3, df_user4, df_user5]

In [10]:
filtered_user1 = df_user1[df_user1['details.state'] == 'DONE'][
    ['userId', 'details.state', 'details.importance', 'details.startDate', 'details.endDate']
]
filtered_user2 = df_user2[df_user2['details.state'] == 'DONE'][
    ['userId', 'details.state', 'details.importance', 'details.startDate', 'details.endDate']
]
filtered_user3 = df_user3[df_user3['details.state'] == 'DONE'][
    ['userId', 'details.state', 'details.importance', 'details.startDate', 'details.endDate']
]
filtered_user4 = df_user4[df_user4['details.state'] == 'DONE'][
    ['userId', 'details.state', 'details.importance', 'details.startDate', 'details.endDate']
]
filtered_user5 = df_user5[df_user5['details.state'] == 'DONE'][
    ['userId', 'details.state', 'details.importance', 'details.startDate', 'details.endDate']
]

In [11]:
filtered_user1

,userId,details.state,details.importance,details.startDate,details.endDate
2,1,DONE,2,2025-06-07T06:09:44.072540,2025-06-10T06:09:44.072540
11,1,DONE,5,2025-06-05T06:09:44.072701,2025-06-12T06:09:44.072701
12,1,DONE,2,2025-06-06T06:09:44.072720,2025-06-12T06:09:44.072720
14,1,DONE,2,2025-05-31T06:09:44.072750,2025-06-15T06:09:44.072750
24,1,DONE,2,2025-06-12T06:09:44.072956,2025-06-16T06:09:44.072956
28,1,DONE,1,2025-05-28T06:09:44.073019,2025-06-12T06:09:44.073019
39,1,DONE,2,2025-06-06T06:09:44.073183,2025-06-12T06:09:44.073183
41,1,DONE,1,2025-06-11T06:09:44.073227,2025-06-15T06:09:44.073227
49,1,DONE,3,2025-06-07T06:09:44.073331,2025-06-10T06:09:44.073331
67,1,DONE,4,2025-06-07T06:09:44.073612,2025-06-11T06:09:44.073612


In [12]:
filtered_user1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 2 to 98
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   userId              19 non-null     object
 1   details.state       19 non-null     object
 2   details.importance  19 non-null     int64 
 3   details.startDate   19 non-null     object
 4   details.endDate     19 non-null     object
dtypes: int64(1), object(4)
memory usage: 912.0+ bytes


In [13]:
# 한 번에 변환 불가 -> 개별 변환
filtered_user1['details.startDate'] = pd.to_datetime(filtered_user1['details.startDate'], utc=True)
filtered_user1['details.endDate'] = pd.to_datetime(filtered_user1['details.endDate'], utc=True)

# 소요 시간 계산
filtered_user1['duration_hours'] = (filtered_user1['details.endDate'] - filtered_user1['details.startDate']).dt.total_seconds() / 3600

filtered_user1

,userId,details.state,details.importance,details.startDate,details.endDate,duration_hours
2,1,DONE,2,2025-06-07 06:09:44.072540+00:00,2025-06-10 06:09:44.072540+00:00,72.0
11,1,DONE,5,2025-06-05 06:09:44.072701+00:00,2025-06-12 06:09:44.072701+00:00,168.0
12,1,DONE,2,2025-06-06 06:09:44.072720+00:00,2025-06-12 06:09:44.072720+00:00,144.0
14,1,DONE,2,2025-05-31 06:09:44.072750+00:00,2025-06-15 06:09:44.072750+00:00,360.0
24,1,DONE,2,2025-06-12 06:09:44.072956+00:00,2025-06-16 06:09:44.072956+00:00,96.0
28,1,DONE,1,2025-05-28 06:09:44.073019+00:00,2025-06-12 06:09:44.073019+00:00,360.0
39,1,DONE,2,2025-06-06 06:09:44.073183+00:00,2025-06-12 06:09:44.073183+00:00,144.0
41,1,DONE,1,2025-06-11 06:09:44.073227+00:00,2025-06-15 06:09:44.073227+00:00,96.0
49,1,DONE,3,2025-06-07 06:09:44.073331+00:00,2025-06-10 06:09:44.073331+00:00,72.0
67,1,DONE,4,2025-06-07 06:09:44.073612+00:00,2025-06-11 06:09:44.073612+00:00,96.0


In [14]:
filtered_user2['details.startDate'] = pd.to_datetime(filtered_user2['details.startDate'], utc=True)
filtered_user2['details.endDate'] = pd.to_datetime(filtered_user2['details.endDate'], utc=True)

filtered_user3['details.startDate'] = pd.to_datetime(filtered_user3['details.startDate'], utc=True)
filtered_user3['details.endDate'] = pd.to_datetime(filtered_user3['details.endDate'], utc=True)

filtered_user4['details.startDate'] = pd.to_datetime(filtered_user4['details.startDate'], utc=True)
filtered_user4['details.endDate'] = pd.to_datetime(filtered_user4['details.endDate'], utc=True)

filtered_user5['details.startDate'] = pd.to_datetime(filtered_user5['details.startDate'], utc=True)
filtered_user5['details.endDate'] = pd.to_datetime(filtered_user5['details.endDate'], utc=True)

# 소요 시간 계산
filtered_user2['duration_hours'] = (filtered_user2['details.endDate'] - filtered_user2['details.startDate']).dt.total_seconds() / 3600
filtered_user3['duration_hours'] = (filtered_user3['details.endDate'] - filtered_user3['details.startDate']).dt.total_seconds() / 3600
filtered_user4['duration_hours'] = (filtered_user4['details.endDate'] - filtered_user4['details.startDate']).dt.total_seconds() / 3600
filtered_user5['duration_hours'] = (filtered_user5['details.endDate'] - filtered_user5['details.startDate']).dt.total_seconds() / 3600

In [15]:
filtered_user1

,userId,details.state,details.importance,details.startDate,details.endDate,duration_hours
2,1,DONE,2,2025-06-07 06:09:44.072540+00:00,2025-06-10 06:09:44.072540+00:00,72.0
11,1,DONE,5,2025-06-05 06:09:44.072701+00:00,2025-06-12 06:09:44.072701+00:00,168.0
12,1,DONE,2,2025-06-06 06:09:44.072720+00:00,2025-06-12 06:09:44.072720+00:00,144.0
14,1,DONE,2,2025-05-31 06:09:44.072750+00:00,2025-06-15 06:09:44.072750+00:00,360.0
24,1,DONE,2,2025-06-12 06:09:44.072956+00:00,2025-06-16 06:09:44.072956+00:00,96.0
28,1,DONE,1,2025-05-28 06:09:44.073019+00:00,2025-06-12 06:09:44.073019+00:00,360.0
39,1,DONE,2,2025-06-06 06:09:44.073183+00:00,2025-06-12 06:09:44.073183+00:00,144.0
41,1,DONE,1,2025-06-11 06:09:44.073227+00:00,2025-06-15 06:09:44.073227+00:00,96.0
49,1,DONE,3,2025-06-07 06:09:44.073331+00:00,2025-06-10 06:09:44.073331+00:00,72.0
67,1,DONE,4,2025-06-07 06:09:44.073612+00:00,2025-06-11 06:09:44.073612+00:00,96.0


In [16]:
time_user1 = filtered_user1[['userId', 'details.importance', 'duration_hours']]
time_user1

,userId,details.importance,duration_hours
2,1,2,72.0
11,1,5,168.0
12,1,2,144.0
14,1,2,360.0
24,1,2,96.0
28,1,1,360.0
39,1,2,144.0
41,1,1,96.0
49,1,3,72.0
67,1,4,96.0


In [17]:
time_user1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 2 to 98
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   userId              19 non-null     object 
 1   details.importance  19 non-null     int64  
 2   duration_hours      19 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 608.0+ bytes


In [18]:
mean_time_user1 = time_user1.groupby(['userId', 'details.importance'])['duration_hours'].mean().reset_index(name='mean_hours')
mean_time_user1

,userId,details.importance,mean_hours
0,1,1,228.0
1,1,2,147.0
2,1,3,216.0
3,1,4,81.0
4,1,5,192.0


In [19]:
time_user2 = filtered_user2[['userId', 'details.importance', 'duration_hours']]
time_user3 = filtered_user3[['userId', 'details.importance', 'duration_hours']]
time_user4 = filtered_user4[['userId', 'details.importance', 'duration_hours']]
time_user5 = filtered_user5[['userId', 'details.importance', 'duration_hours']]

mean_time_user2 = time_user2.groupby(['userId', 'details.importance'])['duration_hours'].mean().reset_index(name='mean_hours')
mean_time_user3 = time_user3.groupby(['userId', 'details.importance'])['duration_hours'].mean().reset_index(name='mean_hours')
mean_time_user4 = time_user4.groupby(['userId', 'details.importance'])['duration_hours'].mean().reset_index(name='mean_hours')
mean_time_user5 = time_user5.groupby(['userId', 'details.importance'])['duration_hours'].mean().reset_index(name='mean_hours')

In [20]:
mean_time_user5

,userId,details.importance,mean_hours
0,5,3,168.0
1,5,5,96.0


In [21]:
json_temp = mean_time_user5.to_dict(orient='records')
# json_temp

with open(f"data\\stat2-user5.json", "w", encoding="utf-8") as f:
    for item in json_temp:  # data는 List[Dict]
        json_line = json.dumps(item, ensure_ascii=False)
        f.write(json_line + "\n")

In [25]:
ptemp = mean_time_user1.to_dict(orient='records')
ptemp

[{'userId': '1', 'details.importance': 1, 'mean_hours': 228.0},
 {'userId': '1', 'details.importance': 2, 'mean_hours': 147.0},
 {'userId': '1', 'details.importance': 3, 'mean_hours': 216.0},
 {'userId': '1', 'details.importance': 4, 'mean_hours': 81.0},
 {'userId': '1', 'details.importance': 5, 'mean_hours': 192.0}]

In [27]:
mean_time_user1
mean_time_user2

,userId,details.importance,mean_hours
0,2,1,115.25
1,2,2,132.00
2,2,3,168.00
3,2,4,108.00
4,2,5,120.00


In [36]:
fig = px.line(
    mean_time_user1,
    x='details.importance',
    y='mean_hours',
    title='Mean Hours by Importance',
    color='userId',
    markers=True,
    labels={
        'details.importance': 'Importance',
        'mean_hours': 'Mean Hours'
    }
)
fig.show()

In [31]:
combined_df = pd.concat([mean_time_user1, mean_time_user2, mean_time_user3, mean_time_user4, mean_time_user5], ignore_index=True)

In [ ]:
fig = px.line(
    combined_df,
    x='details.importance',
    y='mean_hours',
    title='Mean Hours by Importance',
    color='userId',
    markers=True,
    labels={
        'details.importance': 'Importance',
        'mean_hours': 'Mean Hours'
    }
)
fig.show()